In [1]:
import os
import pandas as pd
import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.DataFrame()
for f in [x for x in os.listdir("data") if x[2] == "_"]:
    df = pd.concat([df, pd.read_parquet(os.path.join("data",f))])

In [3]:
df['oscrapovano_minuty'] = df['oscrapovano'].apply(lambda x: str(x)[0:15])

In [4]:
df['oscrapovano_minuty']

0        2024-11-07 18:3
1        2024-11-07 18:3
2        2024-11-07 18:3
3        2024-11-07 18:3
4        2024-11-07 18:3
              ...       
12051    2024-11-14 10:0
12052    2024-11-14 10:0
12053    2024-11-14 16:0
12054    2024-11-14 16:0
12055    2024-11-14 16:0
Name: oscrapovano_minuty, Length: 858672, dtype: object

In [5]:
df = df.drop_duplicates(subset=['odkud','kam','odjezd','oscrapovano_minuty'], keep="last")

In [6]:
df.groupby(['prodejce',pd.Grouper(key='oscrapovano',freq='D')]).size()

prodejce  oscrapovano
ARR       2024-11-07      2497
          2024-11-08      7238
          2024-11-09      3655
          2024-11-10      3242
          2024-11-11      3367
          2024-11-12      3356
          2024-11-13      3657
          2024-11-14      3282
LE        2024-11-07       332
          2024-11-08      1615
          2024-11-09      1873
          2024-11-10      1755
          2024-11-11      1714
          2024-11-12      1906
          2024-11-13      1895
          2024-11-14      1983
RJ        2024-11-06      3877
          2024-11-07      1927
          2024-11-08     12228
          2024-11-09     14652
          2024-11-10     12683
          2024-11-11      7081
          2024-11-12     13862
          2024-11-13     15328
          2024-11-14     11828
ČD        2024-10-31      1558
          2024-11-01     15717
          2024-11-02     21853
          2024-11-03     17590
          2024-11-04     23195
          2024-11-05     22982
          2024-11

In [7]:
df = df.sort_values(by="oscrapovano").reset_index(drop=True)

In [8]:
df = df.dropna(subset=['odkud','kam','odjezd','oscrapovano'],how='any')

In [9]:
df = df[df['prostredek'] != 'autobus']

In [10]:
days = {0: 'po', 1: 'út', 2: 'st', 3: 'čt', 
        4: 'pá', 5: 'so', 6: 'ne'}
df['den'] = df['odjezd'].dt.dayofweek.map(days)

In [11]:
df['predstih_d'] = df['predstih'].dt.days
df['predstih_h'] = df['predstih'].dt.total_seconds() / 3600

In [12]:
df = df[df['predstih_h'] > -3]

In [13]:
kategoricka_data = ['odkud','kam','prodejce','den']

In [14]:
for k in kategoricka_data:
    print("Before:", df[k].memory_usage(deep=True))
    df[k] = df[k].astype('category')
    print("After: ", df[k].memory_usage(deep=True))

Before: 29724093
After:  3668781
Before: 29663304
After:  3667608
Before: 31248455
After:  3660340
Before: 27006392
After:  3660660


In [15]:
df['cena'] = pd.to_numeric(df['cena'])

In [16]:
df.shape

(406660, 26)

In [17]:
df[['odjezd','odkud','kam']].drop_duplicates().shape

(48008, 3)

In [18]:
poradi = ['oscrapovano','prodejce','odkud','kam',
 'odjezd',
 'predstih',
 'predstih_d',
 'predstih_h',
          'cena',
 'prostredek',
          'volnych_mist',
 'obsazenost',
 'jizdni_doba',
 'vzdalenost',
 'zpozdeni',
 'cena_poznamka',
 'den',
 'prestupy',
'vlaky',
 'mistenka_zdarma',
 'nahradni_bus',
 'volna_mista_economy',
 'volna_mista_economy_plus',
 'volna_mista_economy_business',
 'volna_mista_premium']

In [19]:
df[poradi].to_parquet(os.path.join("data","jizdenky.parquet"))

In [20]:
nejnovejsi = df['oscrapovano'].max()
nejnovejsi
df_tyden = df[df['oscrapovano'] > (nejnovejsi - datetime.timedelta(hours=168))]

In [21]:
df_tyden[poradi].to_csv(os.path.join("data","jizdenky_tyden.csv"))